In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [53]:
h100_subcharts = pd.read_csv('hot100_radio_streaming_sales_Kpop.csv')
wdss = pd.read_csv('WDSS_Sentiment_Complete.csv')

## create standardized search_title column for wdss to match other set
wdss['search_title'] = wdss['song_title'].str.replace(r"(\(.*\))", "")
wdss['search_title'] = wdss['search_title'].str.strip()

##match wdss processing to h100/subcharts processing
#clean spacing: add space before linebreaks (so theyre counted separately)
wdss["translated_lyrics"] = wdss["translated_lyrics"].str.replace(r"(\n+)", " \n")
wdss["translated_lyrics"] = wdss["translated_lyrics"].str.replace(r"(\s+)", " ") #restandardize whitespace
wdss["translated_lyrics"] = wdss["translated_lyrics"].str.replace(r"(\s[, | \' | \? | \! | \. | \-]+)", " ")  #delete hanging punctuation
wdss["original_lyrics"] = wdss["original_lyrics"].str.replace(r"(\n+)", " \n")
wdss["original_lyrics"] = wdss["original_lyrics"].str.replace(r"(\s+)", " ") #restandardize whitespace
wdss["original_lyrics"] = wdss["original_lyrics"].str.replace(r"(\s[, | \' | \? | \! | \. | \-]+)", " ")  #delete hanging punctuation

#only roman characters and white space and punctuation
wdss["english_lyrics"] = wdss["original_lyrics"].str.replace(r"([^a-z | A-Z | \s | , | ' | \? | \! | \. | \- ])", " ")
wdss["english_lyrics"] = wdss["english_lyrics"].str.replace(r"(\s+)", " ") #restandardize whitespace
wdss["english_lyrics"] = wdss["english_lyrics"].str.replace(r"(\s[, | ' | \? | \! | \. | \-]+)", " ")  #delete hanging punctuation

##count number of english lyrics and total lyrics
wdss['english_count'] = [len(lyr.split(" ")) for lyr in wdss['english_lyrics']]
wdss['total_count'] = [len(lyr.split(" ")) for lyr in wdss['original_lyrics']]

### create columns for english percentage
h100_subcharts['english_percentage'] = h100_subcharts['english_count'] / h100_subcharts['total_count']
wdss['english_percentage'] = wdss['english_count'] / wdss['total_count']

In [54]:
##test on one song to see if they match
test_title = 'Stay'
h100_value = h100_subcharts.loc[h100_subcharts['search_title'] == test_title]['english_percentage']
wdss_value = wdss.loc[wdss['search_title'] == test_title]['english_percentage']
print('Song ', test_title)
print(h100_value, 'perc. english in h100/subcharts')
print(wdss_value, 'perc. english in wdss')
print('ENGLISH COUNTS:')
print(h100_subcharts.loc[h100_subcharts['search_title'] == test_title]['english_count'], 'h100/subcharts')
print(wdss.loc[wdss['search_title'] == test_title]['english_count'], 'wdss')
print('TOTAL COUNTS:')
print(h100_subcharts.loc[h100_subcharts['search_title'] == test_title]['total_count'], 'h100/subcharts')
print(wdss.loc[wdss['search_title'] == test_title]['total_count'], 'wdss')

Song  Stay
31    0.495726
Name: english_percentage, dtype: float64 perc. english in h100/subcharts
112    0.495726
Name: english_percentage, dtype: float64 perc. english in wdss
ENGLISH COUNTS:
31    116
Name: english_count, dtype: int64 h100/subcharts
112    116
Name: english_count, dtype: int64 wdss
TOTAL COUNTS:
31    234
Name: total_count, dtype: int64 h100/subcharts
112    234
Name: total_count, dtype: int64 wdss


In [55]:
### verify lyrics for both datsets look correct for test song
for i, r in h100_subcharts.iterrows():
    if r['search_title'] == test_title:
        print(r['english_lyrics'])

 heart Together, wherever yeah Wherever yeah I keep sayin' Wherever you are I know you always stay Stay Stay, stay, stay, stay Always Yeah I know you always stay Yeah I know you always stay Yeah I know you always stay Oh every night and day Yeah, I know you always stay We connect to G It ain't the end of the world The present, it shine like a pearl star Wherever you are I know you always stay Stay Stay, stay, stay, stay Always Yeah I know you always stay Yeah I know you always stay Yeah I know you always stay Oh every night and day Yeah, I know you always stay 


In [56]:
for i, r in wdss.iterrows():
    if r['search_title'] == test_title:
        print(r['english_lyrics'])

 heart Together, wherever yeah Wherever yeah I keep sayin' Wherever you are I know you always stay Stay Stay, stay, stay, stay Always Yeah I know you always stay Yeah I know you always stay Yeah I know you always stay Oh every night and day Yeah, I know you always stay We connect to G It ain't the end of the world The present, it shine like a pearl star Wherever you are I know you always stay Stay Stay, stay, stay, stay Always Yeah I know you always stay Yeah I know you always stay Yeah I know you always stay Oh every night and day Yeah, I know you always stay 


In [110]:
#### identify all songs in both datasets and make sure they have matching lyrics info
common_songs = h100_subcharts.merge(wdss, how='inner', on=['search_title'])

In [111]:
target_col = 'total_count'
common_songs.loc[common_songs[target_col+'_y'] != common_songs[target_col+'_x']][['search_title', 'artist_x', 'artist_y', target_col+'_y', target_col+'_x']]

,search_title,artist_x,artist_y,total_count_y,total_count_x


In [112]:
### verify lyrics for both datsets look correct for test song
for i, r in h100_subcharts.iterrows():
    if r['search_title'] == 'Ugh!':
        print(r['original_lyrics'])

Let's go, yeah, yeah, yeah 타닥 또 타오르는 저 불씨 기름에 닿기 전에 먼저 집어삼키네 필시 휩쓸려가겠지 예 예 음 오늘의 선수 입장하시네 건수를, yeah 물기 시작하면 둥둥둥 동네북이 돼 둥둥둥 툭툭 건드네 괜시리 툭툭, yeah 반응이 없음 걍 담궈버리지 푹푹, yeah 진실도 거짓이 돼 거짓도 진실이 돼 이곳에선 모두가 도덕적 사고와 판단이 완벽한 사람이 돼 웃기시네 분노? 물론 필요하지 타오를 땐 이유가 있어 어쩌면 우리의 역사지 그게 세상을 바꾸기도 하지 But 이건 분노 아닌 분뇨 뭐가 분노인지 you know? 분노인 척하며 죽여 진짜 분노 질려버린 수도 없이 많은 people 넌 나만 죽이는 게 아니야 (아니야) 똥 밟는 게 익숙해 우리야 (우리야) 무감각해진 저 사람들 봐 (봐) 분뇨, 무관심 너넨 팀이야, yeah (19115977) 나는 악의에 가득 찬 분노에 분노해 나는 악의에 가득 찬 분노에 분노해 나는 욱해 욱해 나는 욱해 욱해 나는 꺼져야만 했던 분노에 분노해 나는 꺼져야만 했던 그 분노에 분노해 (Yeah, yeah) 그래 욱 욱 욱해라 욱 재가 될 때까지 그래 욱해라 욱 그래 욱 욱 욱해라 욱 부러질 때까지 그래 욱해라 욱 나는 욱해 욱해 나는 욱해 욱해 나는 악의에 가득 찬 분노에 분노해 나는 꺼져야만 했던 그 분노에 분노해 이 세상 분노가 지배함 분노가 없음 다 못 사나 봐 분노하고 또 분노하고 분노하고 그리 미쳐가고 욱 욱 욱 욱 분노하는 이유도 다 수만 가지 선의와 악의도 다 매한가지 분노할 수 있다만 남의 삶에 피해가 있는 건 I don’t like 그건 stop, ayy 누구의 행동에 누구는 아파해 누구의 언행에 누구는 암담해 누구의 찰나에 누구 순간이 돼 누구의 분노에 누구 목숨이 돼 썩을 퉤 나는 욱해 욱해 나는 욱해 욱해 나는 악의에 가득 찬 분노에 분노해 나는 꺼져야만 했던 그 분노에 분노해 아 대체 욕 좀 먹는 게 왜 잘 벌잖아 또 징징대 왜 그 정돈 감수해야지 에헴 에헴 에헴 에헴 에헴

In [113]:
### verify lyrics for both datsets look correct for test song
for i, r in wdss.iterrows():
    if r['search_title'] == 'Ugh!':
        print(r['original_lyrics'])

Let's go, yeah, yeah, yeah 타닥 또 타오르는 저 불씨 기름에 닿기 전에 먼저 집어삼키네 필시 휩쓸려가겠지 예 예 음 오늘의 선수 입장하시네 건수를, yeah 물기 시작하면 둥둥둥 동네북이 돼 둥둥둥 툭툭 건드네 괜시리 툭툭, yeah 반응이 없음 걍 담궈버리지 푹푹, yeah 진실도 거짓이 돼 거짓도 진실이 돼 이곳에선 모두가 도덕적 사고와 판단이 완벽한 사람이 돼 웃기시네 분노? 물론 필요하지 타오를 땐 이유가 있어 어쩌면 우리의 역사지 그게 세상을 바꾸기도 하지 But 이건 분노 아닌 분뇨 뭐가 분노인지 you know? 분노인 척하며 죽여 진짜 분노 질려버린 수도 없이 많은 people 넌 나만 죽이는 게 아니야 (아니야) 똥 밟는 게 익숙해 우리야 (우리야) 무감각해진 저 사람들 봐 (봐) 분뇨, 무관심 너넨 팀이야, yeah (19115977) 나는 악의에 가득 찬 분노에 분노해 나는 악의에 가득 찬 분노에 분노해 나는 욱해 욱해 나는 욱해 욱해 나는 꺼져야만 했던 분노에 분노해 나는 꺼져야만 했던 그 분노에 분노해 (Yeah, yeah) 그래 욱 욱 욱해라 욱 재가 될 때까지 그래 욱해라 욱 그래 욱 욱 욱해라 욱 부러질 때까지 그래 욱해라 욱 나는 욱해 욱해 나는 욱해 욱해 나는 악의에 가득 찬 분노에 분노해 나는 꺼져야만 했던 그 분노에 분노해 이 세상 분노가 지배함 분노가 없음 다 못 사나 봐 분노하고 또 분노하고 분노하고 그리 미쳐가고 욱 욱 욱 욱 분노하는 이유도 다 수만 가지 선의와 악의도 다 매한가지 분노할 수 있다만 남의 삶에 피해가 있는 건 I don’t like 그건 stop, ayy 누구의 행동에 누구는 아파해 누구의 언행에 누구는 암담해 누구의 찰나에 누구 순간이 돼 누구의 분노에 누구 목숨이 돼 썩을 퉤 나는 욱해 욱해 나는 욱해 욱해 나는 악의에 가득 찬 분노에 분노해 나는 꺼져야만 했던 그 분노에 분노해 아 대체 욕 좀 먹는 게 왜 잘 벌잖아 또 징징대 왜 그 정돈 감수해야지 에헴 에헴 에헴 에헴 에헴

In [114]:
# wrong lyrics were identified for gangnam style in wdss....
# h100 is idoll ft. nicki minaj, wdss is idol original version, but they should both be nicki minaj version
# h100 has correct all night lyrics, wdss incorrectly matched it to glory by rm ':|
# ugh! is onlly off by a few words for each dataset so I will let it go

##so basically h100 had correct versions, will overwrite wdss to that

# would be good to do random checks for other wdss songs if time since mismatched songs obviously happened
# (redo it now that ive figured out more accurate way of using genius search??)

In [115]:
## overwrite wdss mistakes with h100 info
to_correct = list(common_songs.loc[common_songs[target_col+'_y'] != common_songs[target_col+'_x']]['search_title'])

for song in to_correct:
    print(song)
    #locate song in each dataset
    h100_index = h100_subcharts.loc[h100_subcharts['search_title'] == song].index[0]
    wdss_index = wdss.loc[wdss['search_title'] == song].index[0]

    ##overwrite wdss info with h100 info
    wdss.at[wdss_index, 'original_lyrics'] = h100_subcharts.at[h100_index, 'original_lyrics']
    wdss.at[wdss_index, 'english_lyrics'] = h100_subcharts.at[h100_index, 'english_lyrics']
    wdss.at[wdss_index, 'translated_lyrics'] = h100_subcharts.at[h100_index, 'translated_lyrics']
    wdss.at[wdss_index, 'english_count'] = h100_subcharts.at[h100_index, 'english_count']
    wdss.at[wdss_index, 'total_count'] = h100_subcharts.at[h100_index, 'total_count']
    wdss.at[wdss_index, 'english_percentage'] = h100_subcharts.at[h100_index, 'english_percentage']

In [116]:
## overwrite each csv with the updated datasets
h100_subcharts.to_csv('hot100_radio_streaming_sales_Kpop.csv')
wdss.to_csv('WDSS_Sentiment_Complete.csv')